In [1]:
import os
import numpy as np
import cv2
import random
from keras import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.applications import vgg16
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import preprocess_input as vgg_pi
from keras.applications.resnet50 import preprocess_input as resnet_pi
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import keras
import tensorflow as tf

In [3]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [4]:
# check if GPU is running
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
img_dir = './data/train/train'
test_dir = "./data/test"
IMG_SIZE = 224
batch_size = 10

In [6]:
train_datagen = ImageDataGenerator(
    #rotation_range=40,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    rescale=1/255,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(
    rescale=1/255,
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=(224,224,),
    classes =["leaf_rust", "stem_rust","healthy_wheat"],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
)
validation_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=(224,224),
    batch_size=batch_size,
    classes =["leaf_rust", "stem_rust","healthy_wheat"],
    class_mode='categorical',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(test_dir, classes=['test'],target_size=(224,224))

Found 701 images belonging to 3 classes.
Found 174 images belonging to 3 classes.
Found 610 images belonging to 1 classes.


## resnet

In [8]:
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False
#base_model.summary()

C:\Users\tetteh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
from keras.layers import GaussianNoise
from keras.regularizers import l2
x = base_model.output

x=  GaussianNoise(0.1)(x)
x = Dropout(0.2)(x)
x = Flatten(name='flatten', input_shape=base_model.output_shape)(x) 
x= Dense(256, activation='relu', name='Dense_Intermediate')(x)
#x = Dropout(0.4)(x)
x = Dense(3,kernel_regularizer=l2(0.01), activation='softmax', name='predictions')(x)
model_resnet = Model(inputs=base_model.input, outputs=x)
model_resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
model_resnet.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
history=model_resnet.fit_generator(train_generator, 
                           steps_per_epoch = len(train_generator.filenames)//batch_size,
                           workers = 4,
                           use_multiprocessing=False,
                           validation_data = validation_generator,
                           validation_steps =len(validation_generator.filenames)//batch_size,
                          
                           epochs =20,
                           #class_weight=class_weights,
                           verbose =2)

Epoch 1/20
 - 85s - loss: 9.5548 - acc: 0.4072 - val_loss: 9.6233 - val_acc: 0.4059
Epoch 2/20
 - 29s - loss: 9.7112 - acc: 0.4001 - val_loss: 9.5694 - val_acc: 0.4085
Epoch 3/20
 - 29s - loss: 8.9243 - acc: 0.4458 - val_loss: 12.7118 - val_acc: 0.1707
Epoch 4/20
 - 28s - loss: 8.3625 - acc: 0.4770 - val_loss: 9.4621 - val_acc: 0.4146
Epoch 5/20
 - 28s - loss: 7.7034 - acc: 0.5156 - val_loss: 10.5228 - val_acc: 0.2500
Epoch 6/20
 - 27s - loss: 7.5218 - acc: 0.5287 - val_loss: 9.4614 - val_acc: 0.4146
Epoch 7/20
 - 28s - loss: 7.3127 - acc: 0.5430 - val_loss: 6.1099 - val_acc: 0.4268
Epoch 8/20
 - 27s - loss: 6.4547 - acc: 0.5943 - val_loss: 9.4593 - val_acc: 0.4146
Epoch 9/20
 - 26s - loss: 6.1836 - acc: 0.6117 - val_loss: 8.5578 - val_acc: 0.2805
Epoch 10/20
 - 26s - loss: 6.3004 - acc: 0.6101 - val_loss: 9.2640 - val_acc: 0.4268
Epoch 11/20
 - 25s - loss: 6.5544 - acc: 0.5928 - val_loss: 9.0672 - val_acc: 0.4390
Epoch 12/20
 - 25s - loss: 7.7657 - acc: 0.5171 - val_loss: 9.4604 - val

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_data=[]
import os
import cv2
from tqdm import tqdm
test1_dir = "./test/test"
def create_test_data():
    path = os.path.join(test1_dir)  
    #class_num = CATEGORIES.index(category)  
    print(path)

    for img in tqdm(os.listdir(path)):  
        try:
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)  # convert to array
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
            test_data.append([new_array])  # add this to our training_data
        except Exception as e:  # in the interest in keeping the output clean...
            pass
            
create_test_data()


testD=np.array(test_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
testD1 = testD/255 
testD.shape


In [ ]:
pred1=model2_vgg16.predict_generator(test_generator)

In [ ]:
pred=model2_vgg16.predict(testD1)

In [ ]:
import pandas as pd

In [ ]:
prediction = pd.DataFrame(pred, columns=["leaf_rust", "stem_rust","healthy_wheat"]).to_csv('./predictions/vgg_l21.csv')